In [6]:
#notes: 
#i can probably just use hsdlink for all the functions
#hsdlink requires device_id

from st_hsdatalog.HSD.HSDatalog import HSDatalog
from st_hsdatalog.HSD_link.HSDLink import HSDLink
from st_hsdatalog.HSD_link.HSDLink_v2 import HSDLink_v2
import matplotlib.pyplot as plt #plotting data

#initializing hsd_link
hsd_link_factory = HSDLink()
hsd_link = hsd_link_factory.create_hsd_link()

hsd_2 = isinstance(hsd_link, HSDLink_v2)

#print(HSDLink.get_version(hsd_link)) #get DLL version

#list of all connected Devices
#dev_list = HSDLink.get_devices(hsd_link)
#for dev in def_list:
#    print(dev)

#select specific device
device_id = 0

device = HSDLink.get_device(hsd_link, device_id)
#print(device)

#device_alias = HSDLink.get_device_alias(hsd_link, device_id)
#print(device_alias)


#Update this boolean to select the right example folder and sensor names in next code cells
hsd_2 = True

#Might remove else case if redundant
if hsd_2:
    acq_folder = "C:\\STWINBOX-Data-Log-Workshop\\STM32CubeFunctionPack_DATALOG2_V1.1.0\\Utilities\\HSDPython_SDK\\examples\\20230818_15_55_41"
else: 
    acq_folder = "C:\\STWINBOX-Data-Log-Workshop\\STM32CubeFunctionPack_DATALOG2_V1.1.0\\Utilities\\HSDPython_SDK\\examples\\STWIN_00001"
    
hsd_factory = HSDatalog()
hsd = hsd_factory.create_hsd(acq_folder)

#get sensors
#sensor_list = HSDatalog.get_sensor_list(hsd, only_active=False) #can also use HSDLink
#HSDatalog.present_sensor_list(hsd, sensor_list)

#get active sensors
#sensor_list = HSDatalog.get_sensor_list(hsd, only_active=True)
#HSDatalog.present_sensor_list(hsd, sensor_list)

#get sensors filtered by type
#acc_list = HSDatalog.get_sensor_list(hsd, type_filter="ACC")
#HSDatalog.present_sensor_list(hsd, acc_list)

#get specific sensor
if hsd_2:
    sensor_name = "ism330dhcx_acc" 
    sensor_name = "ISM330DHCX"

sensor = HSDatalog.get_sensor(hsd, sensor_name)
#HSDatalog.present_sensor(hsd, sensor) #prints info about sensor
#print(sensor) #also prints info but it is not formatted 

s_name = HSDatalog.get_sensor_name(hsd, sensor)
#sensor_enabled = HSDLink.get_sensor_enabled(hsd_link, device id, senosr_name, s_id, ss_id = None) #ss is sub sensor, hsdlink functional for retreiving all sensor details (hsdatalog_communitcation) (device id required)
s_odr = HSDatalog.get_sensor_odr(hsd, sensor)
s_fs = HSDatalog.get_sensor_fs(hsd, sensor)
s_spts = HSDatalog.get_sensor_spts(hsd, sensor)

#IMPORTANT TO DO: hsdlink get methods for sw and hw tag classes found in hsdatalog_communication (not listed)

#print(" --> {} - ODR: {} , FS: {} , SamplesPerTs {}".format(s_name, s_odr, s_fs, s_spts))

#get list of sensor data files in accquision folder
#file_names = HSDatalog.get_dat_file_list(hsd)
#print(file_names)

#extract data from the accquision folder as pandas DataFrames
if hsd_2:
    sensor_name = "ism330dhcx_acc" #needs to have dat file in accquision folder
else:
    sensor_name = "ISM330DHCX"
    
sensor = HSDatalog.get_sensor(hsd, sensor_name)
df = HSDatalog.get_dataframe(hsd, sensor, start_time = 0, end_time = 3, labeled = True, raw_data = False) #start and end times optional, True if data is labeled, False means data will not be multiplied by sensor sensitivity value
#[x.describe() for x in df] #describes stats about dataframe like mean, std, min max, etc.
#[print(x) for x in df] #provides sample of dataframe, same as print(df)

#plotting one sensor
sensor = HSDatalog.get_sensor(hsd, sensor_name)
HSDatalog.plot(hsd, sensor, start_time=0, end_time=10, subplots=False, raw_data=False) #display sub-plot for each dimension (mulidimensional sensors only), raw data is not multiplied by sensitivity

#plot data of all sensors:
#active_sensor_list = HSDatalog.get_all_components(hsd, only_active=True)
#for sensor in active_sensor_list:
#    HSDatalog.plot(hsd, sensor)

#if data is tagged
label_list = HSDatalog.get_acquision_label_classes(hsd)
print(label_list)

HSDatalog.plot(hsd, sensor, label='SW_TAG_0')
HSDatalog.plot(hsd, sensor, label='SW_TAG_2')

#SET single sensor parameter
if hsd_2:
    sensor_name = "ism330dhcx_acc"
    sensor_type = None
    print("--> SET enable Property = True")
    hsd_link.set_sensor_enable(d_id = 0, new_status=True, comp_name=sensor_name)
    print("Enable: {}".format(hsd_link.get_sensor_enable(d_id = 0, sensor_name=sensor_name)))
    print("--> SET odr Property = 1666.0 Hz (DTDL enum=7)")
    hsd_link.set_sensor_odr(d_id = 0, new_odr=7, comp_name=sensor_name)
    print("ODR: {}".format(hsd_link.get_sensor_odr(d_id = 0, sensor_name=sensor_name)))
    print("--> SET fs Property = 8 g (DTDL enum=2)")
    hsd_link.set_sensor_fs(d_id = 0, new_fs=2, comp_name=sensor_name)
    print("FS: {}".format(hsd_link.get_sensor_fs(d_id = 0, sensor_name=sensor_name)))
    print("--> SET SamplePerTimestamp = 500")
    hsd_link.set_sensor_samples_per_ts(d_id = 0, new_spts=500, comp_name=sensor_name)
    print("SamplePerTimestamp: {}".format(hsd_link.get_sensor_samples_per_ts(d_id = 0, sensor_name=sensor_name)))
else:
    print("SensorName: {}".format(hsd_link.get_sensor_name(d_id = 0, s_id = 5)))
    print("SubSensorType: {}".format(hsd_link.get_sub_sensor_type(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET isActive = True")
    hsd_link.set_sub_sensor_active(d_id = 0, s_id = 5, ss_id = 0, new_status = True)
    print("isActive: {}".format(hsd_link.get_sub_sensor_isActive(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET ODR = 3332.0")
    hsd_link.set_sub_sensor_odr(d_id = 0, s_id = 5, ss_id = 0, odr_value = 3332.0)
    print("ODR: {}".format(hsd_link.get_sub_sensor_odr(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET FS = 4.0")
    hsd_link.set_sub_sensor_fs(d_id = 0, s_id = 5, ss_id = 0, fs_value = 4.0)
    print("FS: {}".format(hsd_link.get_sub_sensor_fs(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET SamplePerTimestamp = 1000")
    hsd_link.set_samples_per_timestamp(d_id = 0, s_id = 5, ss_id = 0, spts_value=1000)
    print("SamplePerTimestamp: {}".format(hsd_link.get_sub_sensor_sample_per_ts(d_id = 0, s_id = 5, ss_id = 0)))

#upload ucf file
#if hsd_2:
#    ucf_file_path = "C:\\STWINBOX-Data-Log-Workshop\\STM32CubeFunctionPack_DATALOG2_V1.1.0\\Utilities\\HSDPython_SDK\\examples\\20230818_15_55_41\\create & insert ucf file if applicable"
#else:
#    ucf_file_path = "..\\..\\STWIN_config_examples\\UCF_examples\\ism330dhcx_six_d_position.ucf"
#HSDLink.upload_mlc_ucf_file(hsd_link, device_id, ucf_file_path)

#update config file
if hsd_2:
    json_file_path = "C:\\STWINBOX-Data-Log-Workshop\\STM32CubeFunctionPack_DATALOG2_V1.1.0\\Utilities\\HSDPython_SDK\\examples\\20230818_15_55_41"
else:
    json_file_path = "..\\..\\STWIN_config_examples\\EnvSensors.json"

HSDLink.update_device(hsd_link, device_id, json_file_path)
sensor_list = HSDLink.get_sensor_list(hsd_link, device_id, only_active=False)
HSDatalog.present_sensor_list(hsd, sensor_list)




2023-09-11 17:32:18,271 - HSDatalogApp.st_hsdatalog.HSD_link.communication.STWIN_HSD.hsd_dll - INFO - Windows - 64bit (hsd_dll.py:52)
2023-09-11 17:32:18,287 - HSDatalogApp.st_hsdatalog.HSD_link.communication.STWIN_HSD.STWINHSD_com_manager - INFO - Communication Engine UP (libhs_datalog_v1 DLL/so) (STWINHSD_com_manager.py:135)
2023-09-11 17:32:18,297 - HSDatalogApp.st_hsdatalog.HSD_link.HSDLink_v1 - INFO - STWINHSD_CommandManager command_set: STWINHSDCmd (HSDLink_v1.py:71)
2023-09-11 17:32:18,297 - HSDatalogApp.st_hsdatalog.HSD_link.HSDLink - WARNING - No HSDatalog_v1 devices connected! (HSDLink.py:95)
Trying to open communication using libhs_datalog_v2...
64bit
2023-09-11 17:32:20,300 - HSDatalogApp.st_hsdatalog.HSD_link.communication.PnPL_HSD.PnPLHSD_com_manager - ERROR - Error in Communication Engine opening (libhs_datalog_v2 DLL/so) (PnPLHSD_com_manager.py:35)
2023-09-11 17:32:20,301 - HSDatalogApp.st_hsdatalog.HSD_link.HSDLink - ERROR - Error opening communication using libhs_data

Exception ignored in: <function PnPLHSD_CommandManager.__del__ at 0x0000028DE5148220>
Traceback (most recent call last):
  File "C:\Users\eluru\AppData\Local\Programs\Python\Python311\Lib\site-packages\st_hsdatalog\HSD_link\communication\PnPL_HSD\PnPLHSD_com_manager.py", line 43, in __del__
    raise CommunicationEngineCloseError
st_hsdatalog.HSD_utils.exceptions.CommunicationEngineCloseError: hs_datalog_close


AttributeError: 'NoneType' object has no attribute 'get_device'

In [2]:
import pandas as pd
import numpy as np
import torch

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

#placeholder csv file
csv_file = "C:\\STWINBOX-Data-Log-Workshop\\STM32CubeFunctionPack_DATALOG2_V1.1.0\\Utilities\\HSDPython_SDK\\examples\\20230804_10_27_32_Exported\\ism330dhcx_acc.csv"

########################

#directly convert to tensorflow dataset

#import tensorflow as tf
#col_names = ['Time', 'A_x [g]', 'A_y [g]', 'A_z [g]'] 
#col_types = [tf.float32, tf.float32, tf.float32, tf.float32]
#ataset = tf.data.experimental.make_csv_dataset(csv_file, header=True, batch_size=5) 

#print(dataset)

########################

pandas_df = pd.read_csv(csv_file)

pandas_df.head()

sensor_timestamp = ['Time']
sensor_measures =  ['A_x [g]', 'A_y [g]', 'A_z [g]']

input_np_array = pandas_df[sensor_timestamp].to_numpy
#input_np_array = np.array(pandas_df[sensor_timestamp])
target_np_array = pandas_df[sensor_measures].to_numpy
#input_np_array = np.array(pandas_df[sensor_measures])

#error
t_inputs = torch.tensor(input_np_array, dtype=torch.float32)
t_targets = torch.tensor(target_np_array, dtype=torch.float32)

ds = TensorDataset(t_inputs, t_targets) #torch.utils.data

print(ds)





TypeError: must be real number, not method